## Importación de librerías

In [1]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 73.7 MB/s eta 0:00:00


In [2]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Liberías de optimización
# ==============================================================================
from pulp import *

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

Importación de datos

In [ ]:
from google.colab import files 
files.upload()

In [11]:
# DEMANDA
demanda = pd.read_excel('/content/Prueba_Data.xlsx', sheet_name='DEMANDA_1', index_col=0)
#demanda['DEMANDA'] = demanda['DEMANDA'].astype('int32')

# COSTOS FIJOS CTL
CTL = pd.read_excel('Prueba_Data.xlsx', sheet_name='CTL')
CTL = CTL[['CTL', 'COSTO_FIJO']]

# COSTOS DE TRANSPORTE
costos = pd.read_excel('/content/Prueba_Data.xlsx', sheet_name='COSTOS')

In [12]:
# COSTOS DEL ORIGEN AL PUERTO
costos_puertos = costos[costos.Tipo1.isin(["ORIGEN"]) & costos.Tipo2.isin(["PUERTO"])]
costos_puertos = costos_puertos[['Nodo1_cod', 'Nodo2_cod', 'C_TOTAL']]
costos_puertos.rename(columns={'Nodo1_cod':'ORIGEN', 'Nodo2_cod':'PUERTO'}, inplace=True)
#display(costos_puertos)


# COSTOS DEL PUERTO AL PUNTO DE TRASBORDO
costos_ptm = costos[costos.Tipo1.isin(["PUERTO"]) & costos.Tipo2.isin(["TRASBORDO"])]
costos_ptm = costos_ptm[['Nodo1_cod', 'Nodo2_cod', 'Modo', 'C_TOTAL']]
costos_ptm.rename(columns={'Nodo1_cod':'PUERTO', 'Nodo2_cod':'TRASBORDO', 'Modo':'MODO'}, inplace=True)
costos_ptm.reset_index(inplace=True, drop=True)
costos_ptm['C_TOTAL'] = costos_ptm['C_TOTAL'].astype('int32')
#display(costos_ptm)


# COSTOS DEL PUNTO DE TRASBORDO AL DESTINO

costos_tj = costos[costos.Tipo1.isin(["TRASBORDO"]) & costos.Tipo2.isin(["DESTINO"])]
costos_tj = costos_tj[['Nodo1_cod', 'Nodo2_cod', 'C_TOTAL']]
costos_tj.rename(columns={'Nodo1_cod':'TRASBORDO', 'Nodo2_cod':'DESTINO'}, inplace=True)
costos_tj.reset_index(inplace=True, drop=True)
#costos_tj

Definición de conjuntos y parámetros

In [13]:
# DEFINICIÓN DE CONJUNTOS

origenes = ["MEX", "PAN", "USA", "DEU", "ESP", "BRA"]  # Nodos de origen
destinos = ["MED", "MNZ", "PER", "IBG", "NEV", "BOG", "BUG"]  # Nodos de destino
puertos = ["CTG", "BAQ", "STM"]  # Nodos puertos
trasbordos = ["PLT", "PTB", "DOR", "BCJ", "GRA"]  # Nodos puntos de trasbordo
modos = ["CARRETERO", "FLUVIAL", "FERREO"] # Modos de transporte

In [37]:
# DEFINICIÓN DE PARÁMETROS

capacidad_puerto = {"CTG": 10000000,"BAQ": 10000000,"STM": 10000000}

capacidad_trasbordo = {"PLT": 10000000, 
                       "PTB": 10000000, 
                       "DOR": 10000000, 
                       "BCJ": 10000000, 
                       "GRA": 10000000}

demanda = demanda.transpose().unstack()

costo_origen_puerto = {
    (costos_puertos.ORIGEN[i],costos_puertos.PUERTO[i]): int(costos_puertos.C_TOTAL[i])
    for i in range(len(costos_puertos))
}

costo_puerto_trasbordo = {
    (costos_ptm.PUERTO[i], costos_ptm.TRASBORDO[i], costos_ptm.MODO[i]): int(costos_ptm.C_TOTAL[i])
    for i in range(len(costos_ptm))
}

costo_trasbordo_destino = {
    (costos_tj.TRASBORDO[i],costos_tj.DESTINO[i]): int(costos_tj.C_TOTAL[i])
    for i in range(len(costos_tj))
}

costo_fijo_trasbordo = {
    CTL.CTL[i]: int(CTL.COSTO_FIJO[i])
    for i in range(len(CTL))
}

In [ ]:
#Demanda en formato dict

origenes = demanda.ORIGEN.unique()
destinos = demanda.DESTINO.unique()
DEMANDA_DICT = dict()
type_dict = 'records'

for origen in origenes:
  DEMANDA_DICT[origen] = {}
  for destino in destinos:
    valor = demanda[demanda.ORIGEN.isin([origen]) & demanda.DESTINO.isin([destino])]
    valor.reset_index(inplace=True, drop=True)
    DEMANDA_DICT[origen][destino] = valor.loc[0, 'DEMANDA']

print(DEMANDA_DICT)

In [ ]:
# Diccionario de trasbordos

trasbordos = CTL.CTL.unique()
CTL_DICT = dict()

for trasbordo in trasbordos:
  valor = CTL[CTL.CTL.isin([trasbordo])]
  valor.reset_index(inplace=True, drop=True)
  CTL_DICT[trasbordo] = valor.loc[0, 'COSTO_FIJO']

print(CTL_DICT)

### Prueba 1

In [39]:
# Datos del problema # Parameters
origenes = ['O1', 'O2', 'O3']  # Nodos de origen
destinos = ['D1', 'D2', 'D3']  # Nodos de destino
puertos = ['P1', 'P2', 'P3']  # Nodos puertos
trasbordos = ['T1', 'T2', 'T3']  # Nodos puntos de trasbordo
modos = ['carretero', 'ferreo', 'fluvial']

demanda = {'O1': {'D1': 10, 'D2': 15, 'D3': 20},
           'O2': {'D1': 5, 'D2': 8, 'D3': 12},
           'O3': {'D1': 7, 'D2': 10, 'D3': 15}}

capacidad_puerto = {'P1': 50,'P2': 50,'P3': 50}

capacidad_trasbordo = {'T1': 10000,'T2': 10000, 'T3': 10000}

costo_origen_puerto = {'O1': {'P1': 10, 'P2': 15, 'P3': 12},
                       'O2': {'P1': 8, 'P2': 11, 'P3': 9},
                       'O3': {'P1': 12, 'P2': 10, 'P3': 14}}

costo_puerto_trasbordo = {
    'P1': {'T1': {'carretero': 6, 'ferreo': 7, 'fluvial': 8}, 'T2': {'carretero': 5, 'ferreo': 6, 'fluvial': 7},'T3': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}},
    'P2': {'T1': {'carretero': 8, 'ferreo': 9, 'fluvial': 10}, 'T2': {'carretero': 7, 'ferreo': 8, 'fluvial': 9}, 'T3': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}},
    'P3': {'T1': {'carretero': 9, 'ferreo': 10, 'fluvial': 11}, 'T2': {'carretero': 10, 'ferreo': 11, 'fluvial': 12}, 'T3': {'carretero': 5, 'ferreo': 6, 'fluvial': 7}}
}


costo_trasbordo_destino = {
    'T1': {'D1': 9, 'D2': 12, 'D3': 11},
    'T2': {'D1': 8, 'D2': 10, 'D3': 13},
    'T3': {'D1': 5, 'D2': 13, 'D3': 10}
}

costo_fijo_trasbordo = {
    'T1': 95,
    'T2': 100,
    'T3': 100
}

In [42]:
# Variables
#x = {(i, j, p, t, m): 0 for i in origenes for j in destinos for p in puertos for t in trasbordos for m in modos}
x = LpVariable.dicts('Flujo', (origenes, destinos, puertos, trasbordos, modos),0)

#x = LpVariable.dicts('x', (origenes, destinos, puertos, trasbordos, modos), 0, None, LpContinuous)

# Variable de decisión
#y = LpVariable.dicts('y', (trasbordos), 0, 1, LpBinary)

y = LpVariable.dicts('y', (trasbordos), lowBound = 0, upBound = 1, cat='Integer')

# Model
model = LpProblem("Localizacion de puntos de trasbordo", LpMinimize)
'''
# Objective function
model += lpSum(costo_origen_puerto[i][p]* x[i, j, p, t, m] + 
               costo_puerto_trasbordo[p][t][m]*x[i, j, p, t, m]  + 
               costo_trasbordo_destino[t][j]*x[i, j, p, t, m] + 
               costo_fijo_trasbordo[t]*y[t] for i in origenes for j in destinos for p in puertos for t in trasbordos for m in modos)

model += lpSum(costo_origen_puerto[i][p]* x[i][j][p][t][m] + 
               costo_puerto_trasbordo[p][t][m]*x[i][j][p][t][m] + 
               costo_trasbordo_destino[t][j]*x[i][j][p][t][m] + 
               costo_fijo_trasbordo[t]*y[t] for i in origenes for j in destinos for p in puertos for t in trasbordos for m in modos)
'''
CT = lpSum(costo_origen_puerto[i][p]* x[i][j][p][t][m] + 
           costo_puerto_trasbordo[p][t][m]*x[i][j][p][t][m] + 
           costo_trasbordo_destino[t][j]*x[i][j][p][t][m] +
           costo_fijo_trasbordo[t]*y[t] for i in origenes for j in destinos for p in puertos for t in trasbordos for m in modos)
#CF = lpSum(costo_fijo_trasbordo[t]*y[t] for t in trasbordos)

model += CT

# Constraints
'''
# Restricciones de capacidad en los puertos
for p in puertos:
    model += lpSum(x[i, j, p, t, m] for i in origenes for j in destinos for t in trasbordos for m in modos) <= capacidad_puerto[p]

# Restricciones de capacidad en los trasbordos
for t in trasbordos:
    model += lpSum(x[i, j, p, t, m] for i in origenes for j in destinos for p in puertos for m in modos) <= capacidad_trasbordo[t]

# Restricciones de demanda satisfecha
for i in origenes:
    for j in destinos:
        model += lpSum(x[i, j, p, t, m] for p in puertos for t in trasbordos for m in modos) == demanda[i][j]
'''


# Restricciones de capacidad en los puertos
for p in puertos:
    model += lpSum(x[i][j][p][t][m] for i in origenes for j in destinos for t in trasbordos for m in modos) <= capacidad_puerto[p]

# Restricciones de capacidad en los trasbordos
for t in trasbordos:
    model += lpSum(x[i][j][p][t][m] for i in origenes for j in destinos for p in puertos for m in modos) <= capacidad_trasbordo[t]*y[t]

# Restricciones de demanda satisfecha
for i in origenes:
    for j in destinos:
        model += lpSum(x[i][j][p][t][m] for p in puertos for t in trasbordos for m in modos) == demanda[i][j]

#for j in destinos:
model += lpSum(y[t] for t in trasbordos) == 2 # Cantidad de puertos por seleccionar

# Solve
model.solve()

1

In [43]:
valor = model.objective.value()
print (valor)

# Print results
print("Costo total: ", value(model.objective))
print("-------------------------------------------------")
for t in trasbordos:
        print("Trasbordo: ", t, " Costo fijo: ", costo_fijo_trasbordo[t], " Seleccionado: ", y[t].value())
print("-------------------------------------------------")
for i in origenes:
    for j in destinos:
        for p in puertos:
            for t in trasbordos:
                for m in modos:
                  if x[i][j][p][t][m].value() > 0:
                    print("Origen: ", i, " Destino: ", j, " Puerto: ", p, " Trasbordo: ", t, " Modo de transporte: ", m, " Cantidad: ", x[i][j][p][t][m].value())


18304.0
Costo total:  18304.0
-------------------------------------------------
Trasbordo:  T1  Costo fijo:  95  Seleccionado:  1.0
Trasbordo:  T2  Costo fijo:  100  Seleccionado:  0.0
Trasbordo:  T3  Costo fijo:  100  Seleccionado:  1.0
-------------------------------------------------
Origen:  O1  Destino:  D1  Puerto:  P1  Trasbordo:  T3  Modo de transporte:  carretero  Cantidad:  10.0
Origen:  O1  Destino:  D2  Puerto:  P1  Trasbordo:  T1  Modo de transporte:  carretero  Cantidad:  15.0
Origen:  O1  Destino:  D3  Puerto:  P1  Trasbordo:  T3  Modo de transporte:  carretero  Cantidad:  20.0
Origen:  O2  Destino:  D1  Puerto:  P3  Trasbordo:  T3  Modo de transporte:  carretero  Cantidad:  5.0
Origen:  O2  Destino:  D2  Puerto:  P1  Trasbordo:  T1  Modo de transporte:  carretero  Cantidad:  5.0
Origen:  O2  Destino:  D2  Puerto:  P3  Trasbordo:  T3  Modo de transporte:  carretero  Cantidad:  3.0
Origen:  O2  Destino:  D3  Puerto:  P3  Trasbordo:  T3  Modo de transporte:  carretero  Can